In [53]:
import requests
from bs4 import BeautifulSoup as bs
import re
response = requests.get('http://data.krx.co.kr/contents/MMC/ISIF/isif/MMCISIF003.cmd?tabIndex=0&isuCd=KR7036460004&isuSrtCd=036460&isuTp=STK&isuTpDtl=&prodId=').text
data = bs(response,'html.parser')

print(re.sub('/','>','html/body/div[3]/div[3]/div/div/div[5]/div[5]/div[3]/div/div[1]/div[2]/div[2]/div/div[1]/table/tbody/tr[1]/td[5]/div'))
print(re.sub(r'\[[0-9]\]','','html>body>div[3]>div[3]>div>div>div[5]>div[5]>div[3]>div>div[1]>div[2]>div[2]>div>div[1]>table>tbody>tr[1]>td[5]>div'))

data.select('html>body>div>div>div>div>div>div>div>div>div>div>div>div>div>table>tbody>tr>td>div')


html>body>div[3]>div[3]>div>div>div[5]>div[5]>div[3]>div>div[1]>div[2]>div[2]>div>div[1]>table>tbody>tr[1]>td[5]>div
html>body>div>div>div>div>div>div>div>div>div>div>div>div>div>table>tbody>tr>td>div


[]